In [1]:
import os
import sys
# Use local biomni
sys.path.insert(0, "/home/runai-home/Biomni/")

In [2]:
from biomni.agent import A1
from biomni.config import default_config

from biomni.cogzero.prompt.retriever import RETRIEVER_PROMPT
from biomni.cogzero.prompt.proposer import PROPOSER_PROMPT
from biomni.cogzero.prompt.solver import SOLVER_PROMPT

from biomni.cogzero.canonicalizer import compare_outputs
from biomni.cogzero.utils import clear_persistent_namespace, get_all_resources, parse_solution_as_json, load_data, save_data, load_tasks_as_dict

In [3]:
import random
from pathlib import Path

In [4]:
# RECOMMENDED: Modify global defaults for consistency

# All agents AND database queries use these defaults
default_config.path = "/mlbio_scratch/panigrah/agents/Biomni/data/"
default_config.source = "Custom"
default_config.base_url = "http://localhost:8000/v1"
llm_dir = "/mlbio_scratch/kodryan/hf_downloads/"
# llm_name = "Qwen2.5-Coder-32B-Instruct"
llm_name = "Qwen3-32B"
default_config.llm = os.path.join(llm_dir, llm_name) 
default_config.timeout_seconds = 1200

In [5]:
ver = "v0"
tasks_dir = Path(f"/mlbio_scratch/kodryan/Biomni/data/generated/tasks/{ver}")
clean_tasks_path = tasks_dir / "clean.json"
clean_tasks = load_tasks_as_dict(clean_tasks_path)

### Solver

In [6]:
task = random.choice(clean_tasks)

In [7]:
print(f"Task ID: {task['task_id']}")
print(f"Dataset path: {task['dataset_path']}")
print(f"Dataset description: {task['dataset_description']}")
print(f"Task description: {task['task_description']}")
print(f"Answer: {task['ground_truth_answer']}")

Task ID: 132
Dataset path: /mlbio_scratch/videnovi/papers_BIBA/paper7/data/dataset2.h5ad
Dataset description: scRNA-seq data for human limb
Task description: Using the provided scRNA‑seq AnnData object, compute the proportion of each cell type (column 'cell_type') within each tissue ('tissue' = 'forelimb' or 'hindlimb'), calculate the log2 fold‑change of the forelimb proportion over the hindlimb proportion, and report the cell type with the highest positive log2 fold‑change. Answer format: a single cell‑type label exactly as it appears in the 'cell_type' column (shape: label).
Answer: primitive red blood cell


In [8]:
selected_indices = task["resource_indices"]
solver_agent = A1(resource_indices=selected_indices)

🎓 Academic mode: Using all datasets (including non-commercial)

🔧 BIOMNI CONFIGURATION
📋 DEFAULT CONFIG (Including Database LLM):
  Path: /mlbio_scratch/panigrah/agents/Biomni/data/
  Timeout Seconds: 1200
  Llm: /mlbio_scratch/kodryan/hf_downloads/Qwen3-32B
  Temperature: 0.7
  Use Tool Retriever: True
  Commercial Mode: Academic (all datasets)
  Base Url: http://localhost:8000/v1
  Source: Custom

Checking and downloading missing data lake files...


In [9]:
data_part_prompt = f"Path: {task['dataset_path']}\nDescription: {task['dataset_description']}\n\n"
data_part_prompt = data_part_prompt.strip()

solver_format_dict = {
    "task_description": task["task_description"],
    "data_path": data_part_prompt,
}
solver_prompt = SOLVER_PROMPT.format(**solver_format_dict).strip()

In [10]:
solver_outputs = solver_agent.go(solver_prompt)

Using pre-selected retrieval (no LLM)
================================ Human Message =================================

Using the provided scRNA‑seq AnnData object, compute the proportion of each cell type (column 'cell_type') within each tissue ('tissue' = 'forelimb' or 'hindlimb'), calculate the log2 fold‑change of the forelimb proportion over the hindlimb proportion, and report the cell type with the highest positive log2 fold‑change. Answer format: a single cell‑type label exactly as it appears in the 'cell_type' column (shape: label).

INPUT DATA:
Path: /mlbio_scratch/videnovi/papers_BIBA/paper7/data/dataset2.h5ad
Description: scRNA-seq data for human limb
================================== Ai Message ==================================

<think>
Okay, let's tackle this problem step by step. The user wants me to compute the proportion of each cell type within each tissue (forelimb and hindlimb), then calculate the log2 fold-change of the forelimb proportion over the hindlimb, and fi

<string>:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
/mlbio_scratch/panigrah/miniconda3/envs/biomni_e1/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


================================== Ai Message ==================================

<observation>Cell type with highest positive log2 fold-change: primitive red blood cell
</observation>
================================== Ai Message ==================================

<think>
Okay, let me see. The user asked for the cell type with the highest positive log2 fold-change in proportions between forelimb and hindlimb. The code loaded the AnnData, checked the necessary columns, calculated cell counts per tissue and cell type, then computed the proportions and log2 fold-change. The result was "primitive red blood cell". 

I need to make sure that the code correctly handled the data. Did the groupby and pivot steps work as intended? The code groups by tissue and cell_type to count cells, then merges with total cell counts per tissue. That should give the proportions. Then, the log2 fold-change is calculated correctly as log2(forelimb proportion / hindlimb proportion). The idxmax() function gets 

In [11]:
solver_answer = parse_solution_as_json(solver_outputs[1])
compare_outputs(solver_answer, task["ground_truth_answer"])

Could not parse JSON, returning a raw solution string.


{'equal': True,
 'pred_canon': 'primitive red blood cell',
 'gt_canon': 'primitive red blood cell',
 'diff': ''}

In [12]:
msgs = solver_agent.get_full_trajectory()

In [13]:
for m in msgs:
    print(m["type"])
    print(m["content"])
    print(100 * '-')

system
You are a helpful biomedical assistant assigned with the task of problem-solving.
To achieve this, you will be using an interactive coding environment equipped with a variety of tool functions, data, and softwares to assist you throughout the process.

Given a task, make a plan first. The plan should be a numbered list of steps that you will take to solve the task. Be specific and detailed.
Format your plan as a checklist with empty checkboxes like this:
1. [ ] First step
2. [ ] Second step
3. [ ] Third step

Follow the plan step by step. After completing each step, update the checklist by replacing the empty checkbox with a checkmark:
1. [✓] First step (completed)
2. [ ] Second step
3. [ ] Third step

If a step fails or needs modification, mark it with an X and explain why:
1. [✓] First step (completed)
2. [✗] Second step (failed because...)
3. [ ] Modified second step
4. [ ] Third step

Always show the updated plan after each step so the user can track progress.

At each turn,